In [ ]:
from google.colab import drive
mountpoint = "/content/drive"
drive.mount(mountpoint)

Mounted at /content/drive


In [ ]:

import pandas as pd
import numpy as np
import json
import re

import spacy

from gensim.models import Word2Vec
from scipy import spatial
from scipy.sparse.linalg import svds
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer

!pip install rouge
import rouge
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

In [ ]:
!pip install openpyxl

import pandas as pd
import openpyxl


data_test = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/summarization/billsum_v4_1/sum_case_v0.xlsx')

In [ ]:


data_test.head()

,Unnamed: 0,bill_id,text for summary,summarized_text,clean_summary
0,1,s-1,"On April 8, 1993, BCH Energy, L.P. (¡°BCH¡± or...",This legal case involves a dispute between a w...,NaN
1,2,s-2,Waterscape owns property located in Manhattan....,"On June 28, 2007, Waterscape and Pavarini ente...",NaN
2,3,s-3,NaN,NaN,NaN
3,4,s-4,NaN,NaN,NaN
4,5,s-5,Mandaree Public School District and Tooz Const...,Liberty Mutual was a surety that issued a perf...,NaN


In [ ]:
case = data_test['text for summary']
print(type(case))
case

<class 'pandas.core.series.Series'>


,text for summary
0,"On April 8, 1993, BCH Energy, L.P. (¡°BCH¡± or..."
1,Waterscape owns property located in Manhattan....
2,NaN
3,NaN
4,Mandaree Public School District and Tooz Const...
5,"This case arises under the Miller Act, which r..."
6,This litigation stems from contracts Defendant...
7,"On February 13, 2006, Beech Ridge Energy, LLC ..."
8,The Condominium consists of multiple buildings...
9,"On September 17, 1904, the appellant Memphis T..."


In [ ]:
pip install transformers torch pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel, pipeline


df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/summarization/billsum_v4_1/sum_case_v0.xlsx')

#  "text for summary" 컬럼 데이터 추출
if "text for summary" in df.columns:
    text_data = df["text for summary"].fillna('').tolist()
else:
    raise ValueError("엑셀 파일에서 'text for summary' 열을 찾을 수 없습니다.")

# Legal-BERT 모델 및 요약 모델 로드
legal_bert_model = "nlpaueb/legal-bert-small-uncased"
tokenizer = AutoTokenizer.from_pretrained(legal_bert_model)
model = AutoModel.from_pretrained(legal_bert_model)

# BART 요약 모델
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# 긴 텍스트를 분할
def chunk_text(text, max_length=1024):
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        word_length = len(word) + 1  # 띄어쓰기 포함
        if current_length + word_length > max_length:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0

        current_chunk.append(word)
        current_length += word_length

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

# 요약 실행
summary_results = []
for text in text_data:
    chunks = chunk_text(text)  # 긴 문장 분할
    summaries = [summarizer(chunk, max_length=200, min_length=50, do_sample=False)[0]["summary_text"] for chunk in chunks]
    summary_results.append(" ".join(summaries))

# 결과를 엑셀 파일에 저장
df["summarized_text"] = summary_results
output_path = "summarized_case.xlsx"
df.to_excel("/content/drive/MyDrive/Colab Notebooks/summarization/billsum_v4_1/case_v2_legalBert.xlsx")



Device set to use cuda:0
Your max_length is set to 200, but your input_length is only 193. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=96)
Your max_length is set to 200, but your input_length is only 195. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=97)
Your max_length is set to 200, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 200, but your input_length is only 113. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer

✅ 요약 완료! 결과가 'summarized_case.xlsx' 파일에 저장되었습니다.
